<a href="https://colab.research.google.com/github/0xajinkya/0xajinkya/blob/main/SpamClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [4]:
import pandas as pd
df = pd.read_csv('/content/spam.csv')
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [6]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [7]:
df_spam = df[df['Category'] == 'spam']
df_spam.shape

(747, 2)

In [8]:
df_ham = df[df['Category'] == 'ham']
df_ham.shape

(4825, 2)

In [9]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])

In [10]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced['Category'].value_counts()

spam    747
ham     747
Name: Category, dtype: int64

In [11]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x == 'spam' else 0)

In [12]:
df_balanced.head()

,Category,Message,spam
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",1


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify=df_balanced['spam'])

In [14]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [17]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    '500$ Discount. Hurry Up!',
     "Bhavin, are you up for a volleybal game tomorrow?"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7995451 , -0.4184074 , -0.8055559 , ..., -0.5952295 ,
        -0.7051719 ,  0.8829688 ],
       [-0.87208354, -0.50543964, -0.94446665, ..., -0.85847497,
        -0.71745336,  0.88082975]], dtype=float32)>

In [18]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
cosine_similarity([e[0]], [e[4]])

array([[0.8933632]], dtype=float32)

In [21]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [22]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [23]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=METRICS
)

In [24]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 22s 332ms/step - loss: 0.5956 - accuracy: 0.7080 - precision: 0.6965 - recall: 0.7375
Epoch 2/10
35/35 [==============================] - 12s 338ms/step - loss: 0.4946 - accuracy: 0.8179 - precision: 0.8156 - recall: 0.8214
Epoch 3/10
35/35 [==============================] - 12s 340ms/step - loss: 0.4265 - accuracy: 0.8580 - precision: 0.8463 - recall: 0.8750
Epoch 4/10
35/35 [==============================] - 12s 346ms/step - loss: 0.3814 - accuracy: 0.8946 - precision: 0.8696 - recall: 0.9286
Epoch 5/10
35/35 [==============================] - 12s 350ms/step - loss: 0.3522 - accuracy: 0.8973 - precision: 0.8870 - recall: 0.9107
Epoch 6/10
35/35 [==============================] - 12s 348ms/step - loss: 0.3300 - accuracy: 0.9045 - precision: 0.8807 - recall: 0.9357
Epoch 7/10
35/35 [==============================] - 12s 343ms/step - loss: 0.3127 - accuracy: 0.9009 - precision: 0.8959 - recall: 0.9071
Epoch 8/10
35/35 [================

In [25]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 5s 320ms/step - loss: 0.2995 - accuracy: 0.8850 - precision: 0.8871 - recall: 0.8824


[0.2995411157608032, 0.885026752948761, 0.8870967626571655, 0.8823529481887817]

In [26]:
reviews = [
    'Reply to win Â£100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

1/1 [==============================] - 1s 739ms/step


array([[0.6945444 ],
       [0.74880713],
       [0.7106125 ],
       [0.1536427 ],
       [0.09264851]], dtype=float32)